프로그래밍 언어에 대한 시를 쓰는 데 특화된 체인과 시를 설명하는 데 특화된 체인을 만드세요.


LCEL을 사용해 두 체인을 서로 연결합니다.


최종 체인은 프로그래밍 언어의 이름을 받고 시와 그 설명으로 응답해야 합니다.


모델로는 "gpt-3.5-turbo"를 사용하고 프롬프트에는 ChatPromptTemplate을 사용하세요.

In [6]:
# 필요한 라이브러리 import
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import os

import sys
import os
sys.path.append('../..')
from get_env import get_env

api_key = get_env()

# 모델 초기화
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


In [7]:
# 1. 프로그래밍 언어에 대한 시를 쓰는 체인
poetry_prompt = ChatPromptTemplate.from_template("""
다음 프로그래밍 언어에 대한 아름다운 시를 써주세요: {language}

시는 다음 조건을 만족해야 합니다:
- 4-6줄의 짧은 시
- 프로그래밍 언어의 특징과 철학을 담아야 함
- 감정적이고 시적인 표현 사용
- 한국어로 작성

시:
""")

poetry_chain = poetry_prompt | model | StrOutputParser()


In [8]:
# 2. 시를 설명하는 체인
explanation_prompt = ChatPromptTemplate.from_template("""
다음 시에 대한 상세한 설명을 작성해주세요:

시: {poetry}

설명은 다음 내용을 포함해야 합니다:
- 시의 주제와 메시지
- 사용된 시적 기법과 표현
- 프로그래밍 언어의 어떤 특징이 반영되었는지
- 시의 감정적 효과
- 한국어로 작성

설명:
""")

explanation_chain = explanation_prompt | model | StrOutputParser()


In [9]:
# 3. LCEL을 사용해 두 체인을 연결하는 최종 체인
def format_output(inputs):
    """시와 설명을 포맷팅하는 함수"""
    return f"""
프로그래밍 언어 시

{inputs['poetry']}

시 설명

{inputs['explanation']}
"""

# 올바른 LCEL 체인 구성
final_chain = (
    {"poetry": poetry_chain}
    | RunnablePassthrough.assign(explanation=lambda x: explanation_chain.invoke({"poetry": x["poetry"]}))
    | format_output
)


In [10]:
# 4. 체인 테스트
if __name__ == "__main__":
    # 테스트할 프로그래밍 언어들
    test_languages = ["Python", "JavaScript", "Rust", "Go"]
    
    for lang in test_languages:
        print(f"\n{'='*50}")
        print(f"프로그래밍 언어: {lang}")
        print('='*50)
        
        try:
            result = final_chain.invoke({"language": lang})
            print(result)
        except Exception as e:
            print(f"오류 발생: {e}")
            print("API 키를 확인하고 다시 시도해주세요.")



프로그래밍 언어: Python

프로그래밍 언어 시

파이썬이여, 그 아름다운 언어
간결하고 강력한 표현의 마법
철학은 아름답고 자유로워
코드가 우아하게 흘러 흐르는

들여쓰기로 느끼는 편안함
파이썬이여, 나의 작은 욕망

시 설명

이 시는 파이썬이라는 프로그래밍 언어에 대한 사랑과 감사를 표현한 시로 보입니다. 시인은 파이썬의 간결하고 강력한 표현을 마법으로 묘사하며, 이 언어의 철학이 아름답고 자유롭다고 언급합니다. 또한 코드가 우아하게 흐르는 모습을 통해 파이썬의 우아함과 자연스러움을 강조하고 있습니다.

시의 주제는 파이썬이라는 언어에 대한 사랑과 그 언어가 가지고 있는 매력을 표현한 것으로 보입니다. 시적 기법으로는 음운, 음악적인 구성, 암시, 상징 등이 사용되었을 것으로 예상됩니다. 언어의 특징을 반영하기 위해 시인은 들여쓰기로 느끼는 편안함을 언급하며, 이는 파이썬의 들여쓰기가 코드의 구조를 명확하고 가독성 있게 만들어준다는 점을 강조한 것으로 해석할 수 있습니다.

시의 감정적 효과는 사랑과 열정, 그리고 영감을 주는 언어에 대한 감사와 존경이 느껴집니다. 시인은 파이썬을 자신의 작은 욕망으로 묘사하며, 이를 통해 파이썬에 대한 애정을 표현하고 있습니다. 전체적으로 이 시는 파이썬에 대한 긍정적인 감정과 그 언어가 주는 즐거움을 전달하고자 하는 것으로 해석됩니다.


프로그래밍 언어: JavaScript

프로그래밍 언어 시

자바스크립트야, 너의 유연함이여
변수와 함수를 자유롭게 다루는 그 힘
웹을 빛내는 너의 마력은
감각적인 디자인을 만들어내는 예술가

처음의 복잡함을 극복하고
간결함과 효율을 추구하는 너의 철학
세상을 밝히는 빛이여
자바스크립트야, 너의 아름다움에 경배해

시 설명

이 시는 자바스크립트라는 프로그래밍 언어의 유연성과 강력함을 찬양하는 내용을 담고 있습니다. 시인은 자바스크립트를 변수와 함수를 자유롭게 다루며 웹을 빛내는 마력이라고 표현하고, 그 마력을 통해 감각적인 디자인을 만들어내는 예술가로 묘사합니다.

시적 기법